# Morbilidad Hospitalaria España 2016-2019
## Creación dataframe para visualización

[Datos INE](https://www.ine.es/dyngs/INEbase/es/operacion.htm?c=Estadistica_C&cid=1254736176778&menu=resultados&secc=1254736195291&idp=1254735573175#!tabs-1254736195291)

In [1]:
import pandas as pd
import datetime
import json

In [2]:
#tenemos 4 dataframes por años desde 2016 a 2019
morbi16 = pd.read_csv('../data/morbilidad/EMH_2016_G01.csv',sep='\t')
morbi17 = pd.read_csv('../data/morbilidad/EMH_2017_G01.csv',sep='\t')
morbi18 = pd.read_csv('../data/morbilidad/EMH_2018_G01.csv',sep='\t')
morbi19 = pd.read_csv('../data/morbilidad/EMH_2019_G01.csv',sep='\t')

In [3]:
morbilidad = pd.concat([morbi16,morbi17,morbi18,morbi19])

In [4]:
morbilidad.sample(5)

,Norden,ProvHosp,Sexo,ProvResi,DiagEntr,FxAlta,DiagPrin,MotivoAlta,EdadAnios,EdadMes,EdadDias,DiasEstancia,Factor
844308,37428151,41,1,21,2,11022019,G40409,1,1,2,14,2,1.00000
584326,2612521,4,2,4,2,3012018,D701,1,37,9,10,3,1.00000
293938,23701335,28,2,28,1,18032016,C562,1,43,7,3,10,1.00000
840191,45361424,50,1,50,1,2092016,G478,1,79,4,27,1,1.00000
819965,21940658,28,1,28,2,15072019,G20,1,75,5,25,41,1.01201


In [5]:
#el código de las enferemades mentales empeizan por F (hacemos una selección)
morbilidad = morbilidad[morbilidad["DiagPrin"].str.contains('^F')]
morbilidad.DiagPrin = morbilidad.DiagPrin.str.strip()

In [6]:
morbilidad.drop(['Norden','EdadDias','EdadMes'],axis=1,inplace = True)

In [7]:
morbilidad.DiagPrin.unique()[0:20] #extraer F + 2 primero dígitos para tener los tipos de enfermedasdes mentales agrupadas

array(['F0150', 'F0151', 'F0390', 'F0391', 'F04', 'F05', 'F060', 'F061',
       'F062', 'F0630', 'F0631', 'F0632', 'F0633', 'F0634', 'F064',
       'F068', 'F070', 'F0781', 'F0789', 'F079'], dtype=object)

In [8]:
morbilidad["code"] = morbilidad.DiagPrin.str.extract(r'(^F..)')

In [9]:
morbilidad.code.unique()

array(['F01', 'F03', 'F04', 'F05', 'F06', 'F07', 'F09', 'F10', 'F11',
       'F12', 'F13', 'F14', 'F15', 'F16', 'F17', 'F18', 'F19', 'F20',
       'F21', 'F22', 'F23', 'F24', 'F25', 'F28', 'F29', 'F30', 'F31',
       'F32', 'F33', 'F34', 'F39', 'F40', 'F41', 'F42', 'F43', 'F44',
       'F45', 'F48', 'F50', 'F51', 'F52', 'F53', 'F55', 'F59', 'F60',
       'F63', 'F64', 'F65', 'F66', 'F68', 'F69', 'F70', 'F71', 'F72',
       'F73', 'F78', 'F79', 'F80', 'F81', 'F82', 'F84', 'F88', 'F89',
       'F90', 'F91', 'F93', 'F94', 'F95', 'F98', 'F99'], dtype=object)

## Añadimos códigos CIE (Capítulos y diagnósticos)

In [10]:
#importamos la tabla de correspondencia entre código de diagnóstico agrupadas y nombre
diagnos = pd.read_csv("../data/diagnos.csv")

In [11]:
diagnos.sample(3) 

,code,diag,cap,title
9,F09,Trastornos mentales orgánicos o sintomáticos s...,F00-F09,"Trastornos mentales orgánicos, incluidos los t..."
55,F65,Trastornos de la preferencia sexual,F60-69,Trastornos de la personalidad y del comportami...
23,F23,Trastornos psicóticos agudos y transitorios,F20-29,"Esquizofrenia, trastornos esquizotípicos y tra..."


In [12]:
diagnos.code.unique()

array(['F00', 'F01', 'F02', 'F03', 'F04', 'F05', 'F06', 'F07', 'F08',
       'F09', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16', 'F17',
       'F18', 'F19', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F28',
       'F29', 'F30', 'F31', 'F32', 'F33', 'F34', 'F39', 'F40', 'F41',
       'F42', 'F43', 'F44', 'F45', 'F48', 'F50', 'F51', 'F52', 'F53',
       'F54', 'F55', 'F59', 'F60', 'F62', 'F63', 'F64', 'F66', 'F68',
       'F69', 'F65', 'F70', 'F71', 'F72', 'F73', 'F78', 'F79', 'F80',
       'F81', 'F82', 'F83', 'F84', 'F88', 'F89', 'F90', 'F91', 'F92',
       'F93', 'F94', 'F95', 'F98', 'F99'], dtype=object)

In [13]:
dicc_diag = {} #crear diccionario con diagnósticos y códigos
for i,r in diagnos.iterrows():
    dicc_diag[r.code] = r.diag

In [14]:
dicc_cap = {} #crear diccionario con diagnósticos y códigos
for i,r in diagnos.iterrows():
    dicc_cap[r.code] = r.cap

In [15]:
morbilidad["cap"] = morbilidad.code.map(dicc_cap) #creamos nueva columnas

In [16]:
morbilidad.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,FxAlta,DiagPrin,MotivoAlta,EdadAnios,DiasEstancia,Factor,code,cap
742761,41,1,41,2,17012018,F3110,1,58,18,1.35376,F31,F30-39
744952,28,2,28,1,3082018,F312,1,50,9,1.00000,F31,F30-39
694791,43,1,43,2,16092016,F200,1,24,234,1.00000,F20,F20-29


In [17]:
discapacidad = morbilidad[morbilidad["cap"] == 'F70-79'].index #dropeamos el tipo de retraso mental
morbilidad.drop(discapacidad,inplace = True)

## Creamos fecha de ingreso

In [18]:
#creamos fecha a través de FxAlta
morbilidad.FxAlta = morbilidad.FxAlta.astype('string')

In [19]:
morbilidad['fecha_al'] = pd.to_datetime({'year': morbilidad.FxAlta.str[-4:],
                           'month': morbilidad.FxAlta.str[-6:-4],
                           'day': morbilidad.FxAlta.str[:-6]
                           })

In [20]:
#creamos fecha ingreso restando a fecha_al el número de día ingresado (DiasEstancia)
morbilidad['fecha_ing'] = morbilidad.apply(lambda fila: fila['fecha_al'] - datetime.timedelta(days = fila['DiasEstancia']), axis= 1)

In [21]:
morbilidad['year']=morbilidad.fecha_ing.dt.year
morbilidad['month']=morbilidad.fecha_ing.dt.month
morbilidad['day']=morbilidad.fecha_ing.dt.day

In [22]:
morbilidad.drop(['FxAlta'],axis=1,inplace = True)

In [23]:
# Cambiamos algunos datos categóricos que vienen como números 

In [24]:
morbilidad.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,DiagPrin,MotivoAlta,EdadAnios,DiasEstancia,Factor,code,cap,fecha_al,fecha_ing,year,month,day
713087,13,1,13,2,F200,1,50,10,1.00000,F20,F20-29,2017-07-21,2017-07-11,2017,7,11
725516,38,2,38,2,F314,2,64,30,1.22835,F31,F30-39,2016-08-17,2016-07-18,2016,7,18
744935,28,2,28,1,F312,1,36,21,1.00000,F31,F30-39,2018-05-09,2018-04-18,2018,4,18


In [25]:
#diccionarios códigos
dic_sexo = {1:'Hombre',2:'Mujer'}
dic_entrada = {1:'Ordinario',2:'Urgente'}
dic_alta = {1: "Curación",2:'Traslado',3:'Fallecimiento',4:'Otras'}
num_str = {'1':'01',  '2':'02',  '3':'03',  '4':'04',  '5':'05',  '6':'06',  '7':'07',  '8':'08',  '9':'09'}

In [26]:
morbilidad.ProvHosp = morbilidad.ProvHosp.astype('string').replace(num_str)
morbilidad.ProvResi = morbilidad.ProvResi.astype('string').replace(num_str)

In [27]:
morbilidad['Entrada'] = morbilidad.DiagEntr.map(dic_entrada)
morbilidad['Alta'] = morbilidad.MotivoAlta.map(dic_alta)
morbilidad['SexNom'] = morbilidad.Sexo.map(dic_sexo)

In [28]:
#preparar provincias para geojsaon (nombre)

In [29]:
#diccionario
geo_json = r"../data/socioecon/spain_provinces.geojson"
with open (geo_json) as geo_file:
    provincias = json.load(geo_file)

cod_prov = {}
for i in range(52):    
    cod_prov[provincias['features'][i]['properties']['cod_prov']] = provincias['features'][i]['properties']['name']

In [30]:
morbilidad['ProvHosp_nom'] = morbilidad.ProvHosp.map(cod_prov)
morbilidad['ProvResi_nom'] = morbilidad.ProvResi.map(cod_prov)

In [31]:
#prov resi 53 = extranjeros
morbilidad.ProvResi_nom.fillna('Extranjeros',inplace = True)

In [32]:
morbilidad.isna().sum()

ProvHosp        0
Sexo            0
ProvResi        0
DiagEntr        0
DiagPrin        0
MotivoAlta      0
EdadAnios       0
DiasEstancia    0
Factor          0
code            0
cap             0
fecha_al        0
fecha_ing       0
year            0
month           0
day             0
Entrada         0
Alta            0
SexNom          0
ProvHosp_nom    0
ProvResi_nom    0
dtype: int64

In [33]:
morbilidad.columns

Index(['ProvHosp', 'Sexo', 'ProvResi', 'DiagEntr', 'DiagPrin', 'MotivoAlta',
       'EdadAnios', 'DiasEstancia', 'Factor', 'code', 'cap', 'fecha_al',
       'fecha_ing', 'year', 'month', 'day', 'Entrada', 'Alta', 'SexNom',
       'ProvHosp_nom', 'ProvResi_nom'],
      dtype='object')

In [34]:
morbilidad.rename({'EdadAnios':'Edad','DiasEstancia':'Dias'},axis=1,inplace = True)

In [35]:
morbilidad.sample(3)

,ProvHosp,Sexo,ProvResi,DiagEntr,DiagPrin,MotivoAlta,Edad,Dias,Factor,code,...,fecha_al,fecha_ing,year,month,day,Entrada,Alta,SexNom,ProvHosp_nom,ProvResi_nom
758424,28,1,28,2,F29,1,33,15,1.02096,F29,...,2019-10-31,2019-10-16,2019,10,16,Urgente,Curación,Hombre,Madrid,Madrid
773214,46,1,46,1,F430,1,46,4,1.00000,F43,...,2017-11-09,2017-11-05,2017,11,5,Ordinario,Curación,Hombre,Valencia,Valencia
692224,13,2,13,2,F09,1,79,39,1.00000,F09,...,2017-04-21,2017-03-13,2017,3,13,Urgente,Curación,Mujer,Ciudad Real,Ciudad Real


In [39]:
morbilidad.to_csv('../data/morbilidad_Fgen.csv',index=False)

In [40]:
morbilidad.year.value_counts()

2017    111907
2016    108935
2018    107384
2019    102555
2015      6654
         ...  
1952         1
1975         1
1956         1
1954         1
1961         1
Name: year, Length: 63, dtype: int64

In [41]:
morbilidad['year_al']=morbilidad.fecha_al.dt.year
morbilidad['month_al']=morbilidad.fecha_al.dt.month
morbilidad['day_al']=morbilidad.fecha_al.dt.day

In [55]:
morbilidad.code.value_counts().head(5).keys()

Index(['F20', 'F31', 'F10', 'F60', 'F25'], dtype='object')